In [1]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'racial_covenants_processor.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [2]:
from django.db.models import Count

import pandas as pd
import altair as alt

from apps.deed.models import DeedPage, MatchTerm
from apps.zoon.models import ZooniverseWorkflow, ZooniverseSubject, ManualCovenant
from apps.parcel.utils.export_utils import build_gdf

from django.conf import settings

# 1. Overall confirmation rate

In [3]:
def calculate_confirm_rate(workflow_name, exclude_terms=None):
    workflow = ZooniverseWorkflow.objects.get(workflow_name=workflow_name)
    
    if exclude_terms:
        exclude_kwargs = {
            'matched_terms__term__in': exclude_terms
        }
    else:
        exclude_kwargs = {}
    
    df = pd.DataFrame(
        DeedPage.objects.filter(
            workflow=workflow,
            bool_match=True
        ).exclude(
            zooniverse_subject__isnull=True
        ).exclude(
            zooniverse_subject__bool_handwritten=True
        ).exclude(
            **exclude_kwargs
        ).values(
            'zooniverse_subject__bool_covenant_final'
        ).annotate(
            bool_covenant=Count('zooniverse_subject__bool_covenant_final')
        )
    )
    df['pct_of_total'] = df['bool_covenant'] / df['bool_covenant'].sum()
    return df
    

In [4]:
calculate_confirm_rate('Ramsey County')

,zooniverse_subject__bool_covenant_final,bool_covenant,pct_of_total
0,False,46285,0.950177
1,True,2427,0.049823
2,None,0,0.000000


In [5]:
workflows = ZooniverseWorkflow.objects.all()
for w in workflows:
    print(w.workflow_name)

Ramsey County
MA South Essex County
WI Milwaukee County
MN Dakota County
MN Anoka County
MN Ramsey County
MN Sherburne County
MN Olmsted County
MN Washington County


In [6]:
calculate_confirm_rate('MN Dakota County')

,zooniverse_subject__bool_covenant_final,bool_covenant,pct_of_total
0,False,221,0.107126
1,True,1842,0.892874
2,None,0,0.000000


In [7]:
calculate_confirm_rate('WI Milwaukee County')

,zooniverse_subject__bool_covenant_final,bool_covenant,pct_of_total
0,False,5546,0.492016
1,True,5726,0.507984
2,None,0,0.000000


In [8]:
calculate_confirm_rate('MN Anoka County')

,zooniverse_subject__bool_covenant_final,bool_covenant,pct_of_total
0,False,713,0.55834
1,True,564,0.44166
2,None,0,0.00000


In [9]:
# Milwaukee confirmation rate without "citizen"
calculate_confirm_rate('WI Milwaukee County', ['citizen'])

,zooniverse_subject__bool_covenant_final,bool_covenant,pct_of_total
0,False,2132,0.272008
1,True,5706,0.727992
2,None,0,0.000000


In [10]:
# Anoka confirmation rate without "citizen"
calculate_confirm_rate('WI Milwaukee County', ['citizen'])

,zooniverse_subject__bool_covenant_final,bool_covenant,pct_of_total
0,False,2132,0.272008
1,True,5706,0.727992
2,None,0,0.000000


# 2. Frequency of terms appearing in workflow

In [11]:
def calculate_term_stats_by_doc(workflow_name):
    workflow = ZooniverseWorkflow.objects.get(workflow_name=workflow_name)
    
    # All DeedPage objects
    term_dp_df = pd.DataFrame(DeedPage.objects.filter(workflow=workflow).values('matched_terms__term').annotate(term_count=Count('matched_terms__term')).order_by('-term_count'))
    # term_dp_df

    # Retired subjects
    term_ret_df = pd.DataFrame(DeedPage.objects.filter(workflow=workflow, zooniverse_subject__isnull=False, zooniverse_subject__bool_handwritten=False).values('matched_terms__term').annotate(term_count=Count('matched_terms__term')).order_by('-term_count'))
    # term_ret_df
    
    # Confirmed covenants
    term_cov_df = pd.DataFrame(DeedPage.objects.filter(workflow=workflow, zooniverse_subject__bool_covenant_final=True, zooniverse_subject__bool_handwritten=False).values('matched_terms__term').annotate(term_count=Count('matched_terms__term')).order_by('-term_count'))
    # term_cov_df
    
    workflow_term_df = term_dp_df.rename(columns={'term_count': 'DeedPages'}).merge(
        term_ret_df.rename(columns={'term_count': 'Retired subjects'}),
        how="left",
        on="matched_terms__term"
    ).merge(
        term_cov_df.rename(columns={'term_count': 'Confirmed covenants'}),
        how="left",
        on="matched_terms__term"
    )
    
    cov_docs_count_zoon = ZooniverseSubject.objects.filter(workflow=workflow, bool_covenant_final=True).count()
    cov_docs_count_man = ManualCovenant.objects.filter(workflow=workflow).count()
    
    workflow_term_df.fillna(0, inplace=True)
    workflow_term_df['Confirm rate'] = workflow_term_df['Confirmed covenants'] / workflow_term_df['Retired subjects']
    
    workflow_term_df['Pct of covenants'] = workflow_term_df['Confirmed covenants'] / (cov_docs_count_zoon + cov_docs_count_man)
    
    workflow_term_df = workflow_term_df.sort_values(['Retired subjects'], ascending=False)
    workflow_term_df
    
    return workflow_term_df

In [12]:
dakota_term_stats_by_doc = calculate_term_stats_by_doc('MN Dakota County')
dakota_term_stats_by_doc

,matched_terms__term,DeedPages,Retired subjects,Confirmed covenants,Confirm rate,Pct of covenants
1,negro,820,691.0,682.0,0.986975,0.340489
2,nationality,575,569.0,558.0,0.980668,0.278582
3,japanese,523,477.0,474.0,0.993711,0.236645
4,chinese,505,462.0,457.0,0.989177,0.228158
5,caucasian,466,451.0,444.0,0.984479,0.221667
6,mongolian,416,405.0,403.0,0.995062,0.201198
7,domestic servants,360,357.0,352.0,0.985994,0.175736
8,colored,352,300.0,216.0,0.720000,0.107838
9,african,276,253.0,244.0,0.964427,0.121817
11,turkish,236,234.0,231.0,0.987179,0.115327


In [13]:
def calculate_term_stats_by_lot(workflow_name, terms_list_manual=False, cov_df_manual=False):
    
    
    if isinstance(cov_df_manual, pd.DataFrame):
        workflow = None
        covenanted_lots_df = cov_df_manual[['db_id', 'cov_text']].copy()
    else:
        workflow = ZooniverseWorkflow.objects.get(workflow_name=workflow_name)
        covenanted_lots_df = build_gdf(workflow)[['db_id', 'cov_text']]
    
    if terms_list_manual:
        match_terms = terms_list_manual
    else:
        match_terms = [term for term in DeedPage.objects.filter(workflow=workflow).values_list('matched_terms__term', flat=True).distinct() if term is not None]

    for term in match_terms:
        covenanted_lots_df[term] = covenanted_lots_df['cov_text'].str.contains(term, case=False)
            
    covenanted_lots_df['num_terms'] = covenanted_lots_df[match_terms].eq(True).sum(axis=1)
    
    out_df = pd.DataFrame(columns=['term', 'term_lot_count', 'all_term_lot_count', 'solo_lot_count', 'median_num_terms'])
    for key, term in enumerate(match_terms):
        
        term_lot_count = covenanted_lots_df[covenanted_lots_df[term] == True].shape[0]
        solo_lot_count = covenanted_lots_df[(covenanted_lots_df[term] == True) & (covenanted_lots_df['num_terms'] == 1)].shape[0]
        median_num_terms = covenanted_lots_df[covenanted_lots_df[term] == True]['num_terms'].median() 
        
        new_row = pd.DataFrame({'term': term,
                                'term_lot_count': term_lot_count,
                                'solo_lot_count': solo_lot_count,
                                'median_num_terms': median_num_terms
                            }, index=[key])
        out_df = pd.concat([new_row, out_df.loc[:]]).reset_index(drop=True)
    
    out_df['solo_lot_pct'] = out_df.loc[out_df['solo_lot_count'] > 0].solo_lot_count / out_df.loc[out_df['solo_lot_count'] > 0].term_lot_count
    out_df['all_term_lot_count'] = covenanted_lots_df.shape[0]
    out_df['pct_of_cov_lots'] = out_df['term_lot_count'] / out_df['all_term_lot_count']
    
    out_df = out_df.sort_values(['pct_of_cov_lots'], ascending=False)
    out_df.fillna(0, inplace=True)
    out_df['workflow'] = workflow_name
    return out_df[['workflow', 'term', 'term_lot_count', 'all_term_lot_count', 'solo_lot_count', 'solo_lot_pct', 'median_num_terms', 'pct_of_cov_lots']]

In [14]:
dakota_term_stats_by_lot = calculate_term_stats_by_lot('MN Dakota County')
dakota_term_stats_by_lot

/var/folders/3v/0mvr8p6126n1yv0zq8vn8zyr0000gn/T/ipykernel_54604/3815395501.py:33: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  out_df = pd.concat([new_row, out_df.loc[:]]).reset_index(drop=True)


,workflow,term,term_lot_count,all_term_lot_count,solo_lot_count,solo_lot_pct,median_num_terms,pct_of_cov_lots
15,MN Dakota County,caucasian,456,720,120,0.263158,2.0,0.633333
5,MN Dakota County,domestic servants,318,720,0,0.000000,3.0,0.441667
27,MN Dakota County,nationality,217,720,0,0.000000,3.0,0.301389
2,MN Dakota County,negro,158,720,35,0.221519,3.0,0.219444
1,MN Dakota County,chinese,120,720,0,0.000000,7.0,0.166667
6,MN Dakota County,japanese,118,720,0,0.000000,7.0,0.163889
25,MN Dakota County,mongolian,117,720,0,0.000000,7.0,0.1625
21,MN Dakota County,african,76,720,7,0.092105,7.0,0.105556
10,MN Dakota County,turkish,69,720,0,0.000000,7.0,0.095833
22,MN Dakota County,moorish,67,720,0,0.000000,7.0,0.093056


In [15]:
milw_term_stats_by_lot = calculate_term_stats_by_lot('WI Milwaukee County')
milw_term_stats_by_lot

/var/folders/3v/0mvr8p6126n1yv0zq8vn8zyr0000gn/T/ipykernel_54604/3815395501.py:33: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  out_df = pd.concat([new_row, out_df.loc[:]]).reset_index(drop=True)


,workflow,term,term_lot_count,all_term_lot_count,solo_lot_count,solo_lot_pct,median_num_terms,pct_of_cov_lots
21,WI Milwaukee County,white race,8497,20030,1133,0.133341,2.0,0.424214
9,WI Milwaukee County,domestic servants,7472,20030,286,0.038276,2.0,0.37304
18,WI Milwaukee County,occupied by any,6358,20030,0,0.000000,3.0,0.317424
8,WI Milwaukee County,colored,5959,20030,5229,0.877496,1.0,0.297504
19,WI Milwaukee County,caucasian,4025,20030,632,0.157019,2.0,0.200949
6,WI Milwaukee County,negro,1373,20030,2,0.001457,2.0,0.068547
1,WI Milwaukee County,ethiopian,1200,20030,0,0.000000,2.0,0.05991
4,WI Milwaukee County,person not of,601,20030,0,0.000000,2.0,0.030005
32,WI Milwaukee County,mongolian,363,20030,0,0.000000,2.0,0.018123
22,WI Milwaukee County,citizen,240,20030,0,0.000000,3.0,0.011982


In [16]:
milw_term_stats_by_doc = calculate_term_stats_by_doc('WI Milwaukee County')
milw_term_stats_by_doc

,matched_terms__term,DeedPages,Retired subjects,Confirmed covenants,Confirm rate,Pct of covenants
1,citizen,6263,3430.0,20.0,0.005831,0.003436
2,colored,6117,3382.0,3180.0,0.940272,0.546392
3,white race,2652,1601.0,1596.0,0.996877,0.274227
0,occupied by any,7289,1064.0,1060.0,0.996241,0.182131
4,polish,1683,635.0,1.0,0.001575,0.000172
5,caucasian,999,618.0,612.0,0.990291,0.105155
6,domestic servants,830,478.0,475.0,0.993724,0.081615
7,jewish,714,425.0,2.0,0.004706,0.000344
8,african,655,268.0,192.0,0.716418,0.032990
9,italian,378,206.0,3.0,0.014563,0.000515


In [38]:
anoka_term_stats_by_doc = calculate_term_stats_by_doc('MN Anoka County')
anoka_term_stats_by_doc

,matched_terms__term,DeedPages,Retired subjects,Confirmed covenants,Confirm rate,Pct of covenants
1,caucasian,1142,406.0,353.0,0.869458,0.605489
2,indian,990,298.0,3.0,0.010067,0.005146
3,colored,577,195.0,155.0,0.794872,0.265866
4,negro,552,190.0,176.0,0.926316,0.301887
5,alien,492,121.0,0.0,0.000000,0.000000
8,polish,131,55.0,0.0,0.000000,0.000000
7,racial,133,49.0,29.0,0.591837,0.049743
0,citizen,3904,48.0,0.0,0.000000,0.000000
9,person not of,86,28.0,28.0,1.000000,0.048027
10,persian,85,28.0,0.0,0.000000,0.000000


In [17]:
anoka_term_stats_by_lot = calculate_term_stats_by_lot('MN Anoka County')
anoka_term_stats_by_lot

/var/folders/3v/0mvr8p6126n1yv0zq8vn8zyr0000gn/T/ipykernel_54604/3815395501.py:33: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  out_df = pd.concat([new_row, out_df.loc[:]]).reset_index(drop=True)


,workflow,term,term_lot_count,all_term_lot_count,solo_lot_count,solo_lot_pct,median_num_terms,pct_of_cov_lots
17,MN Anoka County,caucasian,633,780,313,0.494471,2.0,0.811538
1,MN Anoka County,person not of,126,780,0,0.000000,2.0,0.161538
7,MN Anoka County,domestic servants,124,780,0,0.000000,2.0,0.158974
16,MN Anoka County,occupied by any,115,780,0,0.000000,2.0,0.147436
4,MN Anoka County,negro,112,780,4,0.035714,2.0,0.14359
6,MN Anoka County,colored,108,780,2,0.018519,2.0,0.138462
9,MN Anoka County,racial,27,780,27,1.000000,1.0,0.034615
19,MN Anoka County,white race,6,780,6,1.000000,1.0,0.007692
24,MN Anoka County,african,2,780,0,0.000000,4.0,0.002564
0,MN Anoka County,chinese,2,780,0,0.000000,4.0,0.002564


In [18]:
all_terms = list(MatchTerm.objects.exclude(term=None).distinct().values_list('term', flat=True))
sorted(all_terms)

[' alien',
 ' aryan',
 ' indian',
 ' indian ',
 ' jew ',
 ' malay',
 ' not white',
 ' polish',
 'african',
 'armenian',
 'blood group',
 'caucasian',
 'caucasion',
 'caucausian',
 'certificate of death',
 'chinese',
 'citizen',
 'colored',
 'date of death',
 'death certificate',
 'domestic servants',
 'ethiopian',
 'hebrew',
 'hindu',
 'irish',
 'italian',
 'japanese',
 'jewish',
 'mexican',
 'mongolian',
 'moorish',
 'mulato',
 'mulatto',
 'nationality',
 'negro',
 'occupied by any',
 'persian',
 'person not of',
 'persons not of',
 'racial',
 'report of transfer',
 'semetic',
 'semitic',
 'simitic',
 'syrian',
 'transfer or discharge',
 'turkish',
 'white race']

In [19]:
hennepin_cov_df = pd.read_csv('https://github.com/UMNLibraries/mp-us-racial-covenants/raw/main/mn-hennepin-county/covenants-mn-hennepin-county.csv')
hennepin_cov_df.head()

/var/folders/3v/0mvr8p6126n1yv0zq8vn8zyr0000gn/T/ipykernel_54604/3968416624.py:1: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  hennepin_cov_df = pd.read_csv('https://github.com/UMNLibraries/mp-us-racial-covenants/raw/main/mn-hennepin-county/covenants-mn-hennepin-county.csv')


,db_id,workflow,cnty_name,cnty_fips,doc_num,deed_year,deed_date,exec_date,cov_text,seller,...,geocd_dist,cov_type,match_type,manual_cx,dt_updated,zn_subj_id,zn_dt_ret,image_ids,med_score,plat_dbid
0,0,Zooniverse -- Torrens,Hennepin,27053,T00155264,1940,NaN,1940-03-23,Subject to certain racial and building restric...,James E. O'Brien and Agnes B. O'Brien,...,4.563108,zooniverse,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Zooniverse -- Torrens,Hennepin,27053,T00220683,1946,NaN,1946-04-02,no persons of any race other than of the Cauca...,Rudolph C. & Opal M. Soens,...,0.000000,zooniverse,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Zooniverse -- Torrens,Hennepin,27053,T00220683,1946,NaN,1946-04-02,no persons of any race other than of the Cauca...,Rudolph C. & Opal M. Soens,...,0.200013,zooniverse,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Zooniverse -- Torrens,Hennepin,27053,T00219828,1946,NaN,1946-03-01,racial restrictions of record,A. G. Bogen Company,...,0.000000,zooniverse,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Zooniverse -- Torrens,Hennepin,27053,T00220683,1946,NaN,1946-04-02,no persons of any race other than of the Cauca...,Rudolph C. & Opal M. Soens,...,0.000000,zooniverse,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
hennepin_term_stats_by_lot = calculate_term_stats_by_lot('MN Hennepin County', all_terms, hennepin_cov_df)
hennepin_term_stats_by_lot

/var/folders/3v/0mvr8p6126n1yv0zq8vn8zyr0000gn/T/ipykernel_54604/3815395501.py:33: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  out_df = pd.concat([new_row, out_df.loc[:]]).reset_index(drop=True)


,workflow,term,term_lot_count,all_term_lot_count,solo_lot_count,solo_lot_pct,median_num_terms,pct_of_cov_lots
18,MN Hennepin County,caucasian,16461,24118,10510,0.638479,1.0,0.682519
6,MN Hennepin County,colored,3766,24118,3403,0.903611,1.0,0.156149
17,MN Hennepin County,occupied by any,3519,24118,36,0.010230,2.0,0.145908
7,MN Hennepin County,domestic servants,2739,24118,0,0.000000,2.0,0.113567
4,MN Hennepin County,negro,2007,24118,330,0.164425,7.0,0.083216
32,MN Hennepin County,african,1925,24118,399,0.207273,7.0,0.079816
46,MN Hennepin County,chinese,1486,24118,0,0.000000,7.0,0.061614
38,MN Hennepin County,mongolian,1445,24118,0,0.000000,7.0,0.059914
35,MN Hennepin County,moorish,1416,24118,0,0.000000,7.0,0.058711
8,MN Hennepin County,japanese,1371,24118,0,0.000000,7.0,0.056846


In [21]:
ramsey_term_stats_by_lot = calculate_term_stats_by_lot('Ramsey County', all_terms)
ramsey_term_stats_by_lot

/var/folders/3v/0mvr8p6126n1yv0zq8vn8zyr0000gn/T/ipykernel_54604/3815395501.py:33: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  out_df = pd.concat([new_row, out_df.loc[:]]).reset_index(drop=True)


,workflow,term,term_lot_count,all_term_lot_count,solo_lot_count,solo_lot_pct,median_num_terms,pct_of_cov_lots
18,Ramsey County,caucasian,2282,5458,1035,0.453550,2.0,0.418102
6,Ramsey County,colored,1749,5458,1564,0.894225,1.0,0.320447
17,Ramsey County,occupied by any,1052,5458,4,0.003802,2.0,0.192745
4,Ramsey County,negro,774,5458,55,0.071059,6.0,0.14181
32,Ramsey County,african,631,5458,199,0.315372,7.0,0.11561
46,Ramsey County,chinese,571,5458,0,0.000000,7.0,0.104617
8,Ramsey County,japanese,570,5458,0,0.000000,7.0,0.104434
7,Ramsey County,domestic servants,558,5458,0,0.000000,2.0,0.102235
38,Ramsey County,mongolian,458,5458,0,0.000000,7.0,0.083914
13,Ramsey County,turkish,399,5458,0,0.000000,7.0,0.073104


In [22]:
combined_stats_by_lot = pd.concat([hennepin_term_stats_by_lot, ramsey_term_stats_by_lot, dakota_term_stats_by_lot, milw_term_stats_by_lot, anoka_term_stats_by_lot], axis=0) 

In [23]:
alt.Chart(
    combined_stats_by_lot[combined_stats_by_lot['term_lot_count'] > 0].sort_values('pct_of_cov_lots', ascending=False),
    title="Percent of covenanted lots with each term, by County"
).mark_bar().encode(
   x=alt.X('term', axis=alt.Axis(labelAngle=0)).sort("-y"),
   xOffset='workflow',
   y=alt.Y('pct_of_cov_lots', axis=alt.Axis(grid=False)),
   color='workflow'
).configure_view(
    stroke=None,
)

/Users/mcorey/.local/share/virtualenvs/racial_covenants_processor-wniTpM7j/lib/python3.9/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/mcorey/.local/share/virtualenvs/racial_covenants_processor-wniTpM7j/lib/python3.9/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/mcorey/.local/share/virtualenvs/racial_covenants_processor-wniTpM7j/lib/python3.9/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).ap

alt.Chart(...)

In [24]:
def build_term_freq_chart(df):
    return alt.Chart(df[df['term_lot_count'] > 0].sort_values('pct_of_cov_lots', ascending=False), title=df.iloc[0].workflow).mark_bar().encode(
        x=alt.X('term').sort("-y"),
        y='pct_of_cov_lots'
    )

In [25]:
combined_stats_by_lot = pd.concat([
    hennepin_term_stats_by_lot,
    ramsey_term_stats_by_lot,
    dakota_term_stats_by_lot,
    milw_term_stats_by_lot, 
    anoka_term_stats_by_lot
])
combined_stats_by_lot

,workflow,term,term_lot_count,all_term_lot_count,solo_lot_count,solo_lot_pct,median_num_terms,pct_of_cov_lots
18,MN Hennepin County,caucasian,16461,24118,10510,0.638479,1.0,0.682519
6,MN Hennepin County,colored,3766,24118,3403,0.903611,1.0,0.156149
17,MN Hennepin County,occupied by any,3519,24118,36,0.010230,2.0,0.145908
7,MN Hennepin County,domestic servants,2739,24118,0,0.000000,2.0,0.113567
4,MN Hennepin County,negro,2007,24118,330,0.164425,7.0,0.083216
...,...,...,...,...,...,...,...,...
10,MN Anoka County,syrian,0,780,0,0.000000,0.0,0.0
5,MN Anoka County,polish,0,780,0,0.000000,0.0,0.0
3,MN Anoka County,not white,0,780,0,0.000000,0.0,0.0
2,MN Anoka County,hindu,0,780,0,0.000000,0.0,0.0


In [26]:
grouped_stats_by_lot = combined_stats_by_lot[[
    'term',
    'term_lot_count',
    'all_term_lot_count',
    'solo_lot_count'
]].groupby(['term']).agg('sum').sort_values('term_lot_count', ascending=False).reset_index()
grouped_stats_by_lot['all_term_lot_count'] = grouped_stats_by_lot['all_term_lot_count'].max()
grouped_stats_by_lot = grouped_stats_by_lot[grouped_stats_by_lot['term_lot_count'] > 0]
grouped_stats_by_lot['pct_of_cov_lots'] = grouped_stats_by_lot['term_lot_count'] / grouped_stats_by_lot['all_term_lot_count']
grouped_stats_by_lot['solo_lot_pct'] = grouped_stats_by_lot['solo_lot_count'] / grouped_stats_by_lot['term_lot_count']
grouped_stats_by_lot

,term,term_lot_count,all_term_lot_count,solo_lot_count,pct_of_cov_lots,solo_lot_pct
0,caucasian,23857,51106,12610,0.466814,0.528566
1,colored,11641,51106,10209,0.227781,0.876987
2,domestic servants,11211,51106,286,0.219368,0.025511
3,occupied by any,11104,51106,40,0.217274,0.003602
4,white race,9125,51106,1678,0.17855,0.18389
5,negro,4424,51106,426,0.086565,0.096293
6,african,2805,51106,766,0.054886,0.273084
7,mongolian,2383,51106,0,0.046629,0.0
8,chinese,2179,51106,0,0.042637,0.0
9,japanese,2061,51106,0,0.040328,0.0


In [27]:
grouped_stats_by_lot_display = grouped_stats_by_lot.copy().drop(columns=['all_term_lot_count', 'solo_lot_count'])
grouped_stats_by_lot_display = grouped_stats_by_lot_display.sort_values('pct_of_cov_lots', ascending=False)
grouped_stats_by_lot_display['Rank'] = grouped_stats_by_lot_display.index + 1
grouped_stats_by_lot_display['term_lot_count'] = grouped_stats_by_lot_display['term_lot_count'].apply(lambda x: f"{x:,}")
grouped_stats_by_lot_display['pct_of_cov_lots'] = grouped_stats_by_lot_display['pct_of_cov_lots'].apply(lambda x: f"{x:.1%}")
grouped_stats_by_lot_display['solo_lot_pct'] = grouped_stats_by_lot_display['solo_lot_pct'].apply(lambda x: f"{x:.1%}")
grouped_stats_by_lot_display = grouped_stats_by_lot_display[[
    'Rank', 'term', 'term_lot_count', 'pct_of_cov_lots', 'solo_lot_pct'
]]
grouped_stats_by_lot_display.rename(columns={
    'term': 'Term',
    'term_lot_count': 'Covenanted lot count',
    'pct_of_cov_lots': 'Percent of covenanted lots',
    'solo_lot_pct': 'Solo frequency',
}, inplace=True)

# out_dir = os.path.join(settings.BASE_DIR, '/nostat_exports')
grouped_stats_by_lot_display.to_csv('overall_frequency.csv', index=False)
grouped_stats_by_lot_display

,Rank,Term,Covenanted lot count,Percent of covenanted lots,Solo frequency
0,1,caucasian,"23,857",46.7%,52.9%
1,2,colored,"11,641",22.8%,87.7%
2,3,domestic servants,"11,211",21.9%,2.6%
3,4,occupied by any,"11,104",21.7%,0.4%
4,5,white race,"9,125",17.9%,18.4%
5,6,negro,"4,424",8.7%,9.6%
6,7,african,"2,805",5.5%,27.3%
7,8,mongolian,"2,383",4.7%,0.0%
8,9,chinese,"2,179",4.3%,0.0%
9,10,japanese,"2,061",4.0%,0.0%


In [28]:
for df in [hennepin_term_stats_by_lot, ramsey_term_stats_by_lot, dakota_term_stats_by_lot, milw_term_stats_by_lot, anoka_term_stats_by_lot]:
    chart = build_term_freq_chart(df)
    display(chart)

/Users/mcorey/.local/share/virtualenvs/racial_covenants_processor-wniTpM7j/lib/python3.9/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/mcorey/.local/share/virtualenvs/racial_covenants_processor-wniTpM7j/lib/python3.9/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/mcorey/.local/share/virtualenvs/racial_covenants_processor-wniTpM7j/lib/python3.9/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).ap

alt.Chart(...)

/Users/mcorey/.local/share/virtualenvs/racial_covenants_processor-wniTpM7j/lib/python3.9/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/mcorey/.local/share/virtualenvs/racial_covenants_processor-wniTpM7j/lib/python3.9/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/mcorey/.local/share/virtualenvs/racial_covenants_processor-wniTpM7j/lib/python3.9/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).ap

alt.Chart(...)

/Users/mcorey/.local/share/virtualenvs/racial_covenants_processor-wniTpM7j/lib/python3.9/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/mcorey/.local/share/virtualenvs/racial_covenants_processor-wniTpM7j/lib/python3.9/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/mcorey/.local/share/virtualenvs/racial_covenants_processor-wniTpM7j/lib/python3.9/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).ap

alt.Chart(...)

/Users/mcorey/.local/share/virtualenvs/racial_covenants_processor-wniTpM7j/lib/python3.9/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/mcorey/.local/share/virtualenvs/racial_covenants_processor-wniTpM7j/lib/python3.9/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/mcorey/.local/share/virtualenvs/racial_covenants_processor-wniTpM7j/lib/python3.9/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).ap

alt.Chart(...)

/Users/mcorey/.local/share/virtualenvs/racial_covenants_processor-wniTpM7j/lib/python3.9/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/mcorey/.local/share/virtualenvs/racial_covenants_processor-wniTpM7j/lib/python3.9/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/mcorey/.local/share/virtualenvs/racial_covenants_processor-wniTpM7j/lib/python3.9/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).ap

alt.Chart(...)

In [29]:
def calculate_term_stats_by_year(workflow_name, terms_list_manual=False, cov_df_manual=False, exclude_terms=None):
    
    
    if isinstance(cov_df_manual, pd.DataFrame):
        workflow = None
        covenanted_lots_df = cov_df_manual[['db_id', 'cov_text', 'deed_year']].copy().fillna(0)
    else:
        workflow = ZooniverseWorkflow.objects.get(workflow_name=workflow_name)
        covenanted_lots_df = build_gdf(workflow)[['db_id', 'cov_text', 'deed_year']]
    
    if terms_list_manual:
        match_terms = terms_list_manual
    else:
        match_terms = [term for term in DeedPage.objects.filter(workflow=workflow).values_list('matched_terms__term', flat=True).distinct() if term is not None]

    if exclude_terms:
        match_terms = [term for term in match_terms if term not in exclude_terms]

    for term in match_terms:
        covenanted_lots_df[term] = covenanted_lots_df['cov_text'].str.contains(term, case=False)
    
    out_df = pd.DataFrame(columns=['term', 'year', 'term_lot_count'])
    for key, term in enumerate(match_terms):
        lot_counts = covenanted_lots_df[covenanted_lots_df[term] == True][
            ['db_id', 'deed_year']
        ].groupby(['deed_year']).agg('count').reset_index().rename(columns={
            'deed_year': 'year',
            'db_id': 'term_lot_count'
        })
        lot_counts['term'] = term
        lot_counts['workflow'] = workflow_name

        out_df = pd.concat([lot_counts, out_df.loc[:]]).reset_index(drop=True)
    out_df.fillna(0, inplace=True)
    return out_df[['workflow', 'term', 'year', 'term_lot_count']]


In [30]:
def build_term_freq_chart(df, num_terms=10):
    if num_terms:
        term_counts = df[['term', 'term_lot_count']].groupby(['term']).agg('sum').reset_index().sort_values('term_lot_count', ascending=False)
        df = df[df['term'].isin(term_counts['term'][0:num_terms])]

    return alt.Chart(df[df['term_lot_count'] > 0].sort_values('term_lot_count', ascending=False), title=df.iloc[0].workflow).mark_line().encode(
        x=alt.X('year:O'),
        y='term_lot_count',
        color='term'
    )

In [31]:
exclude_terms = ['occupied by any', 'domestic servants']

In [32]:
milw_term_stats_by_year = calculate_term_stats_by_year('WI Milwaukee County', False, False, exclude_terms)
build_term_freq_chart(milw_term_stats_by_year, 5)

/var/folders/3v/0mvr8p6126n1yv0zq8vn8zyr0000gn/T/ipykernel_54604/1316910966.py:33: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  out_df = pd.concat([lot_counts, out_df.loc[:]]).reset_index(drop=True)
/Users/mcorey/.local/share/virtualenvs/racial_covenants_processor-wniTpM7j/lib/python3.9/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/mcorey/.local/share/virtualenvs/racial_covenants_processor-wniTpM7j/lib/python3.9/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is de

alt.Chart(...)

In [33]:
ramsey_term_stats_by_year = calculate_term_stats_by_year('Ramsey County', all_terms, False, exclude_terms)
build_term_freq_chart(ramsey_term_stats_by_year, 5)

/var/folders/3v/0mvr8p6126n1yv0zq8vn8zyr0000gn/T/ipykernel_54604/1316910966.py:33: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  out_df = pd.concat([lot_counts, out_df.loc[:]]).reset_index(drop=True)
/Users/mcorey/.local/share/virtualenvs/racial_covenants_processor-wniTpM7j/lib/python3.9/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/mcorey/.local/share/virtualenvs/racial_covenants_processor-wniTpM7j/lib/python3.9/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is de

alt.Chart(...)

In [34]:
hennepin_term_stats_by_year = calculate_term_stats_by_year('MN Hennepin County', all_terms, hennepin_cov_df, exclude_terms)
hennepin_term_stats_by_year
build_term_freq_chart(hennepin_term_stats_by_year, 5)

/Users/mcorey/.local/share/virtualenvs/racial_covenants_processor-wniTpM7j/lib/python3.9/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [35]:
dakota_term_stats_by_year = calculate_term_stats_by_year('MN Dakota County', False, False, exclude_terms)
build_term_freq_chart(dakota_term_stats_by_year, 5)

/var/folders/3v/0mvr8p6126n1yv0zq8vn8zyr0000gn/T/ipykernel_54604/1316910966.py:33: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  out_df = pd.concat([lot_counts, out_df.loc[:]]).reset_index(drop=True)
/Users/mcorey/.local/share/virtualenvs/racial_covenants_processor-wniTpM7j/lib/python3.9/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/mcorey/.local/share/virtualenvs/racial_covenants_processor-wniTpM7j/lib/python3.9/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is de

alt.Chart(...)

In [36]:
anoka_term_stats_by_year = calculate_term_stats_by_year('MN Anoka County', False, False, exclude_terms)
build_term_freq_chart(anoka_term_stats_by_year, 5)

/Users/mcorey/.local/share/virtualenvs/racial_covenants_processor-wniTpM7j/lib/python3.9/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

# Combined stats by doc (confirmation rate chart for NEH white paper)

In [47]:
combined_stats_by_doc = pd.concat([
    dakota_term_stats_by_doc,
    milw_term_stats_by_doc, 
    anoka_term_stats_by_doc
])
combined_stats_by_doc.rename(columns={'matched_terms__term': 'Term'}, inplace=True)
combined_stats_by_doc = combined_stats_by_doc[combined_stats_by_doc['Retired subjects'] > 0]
combined_stats_by_doc

,Term,DeedPages,Retired subjects,Confirmed covenants,Confirm rate,Pct of covenants
1,negro,820,691.0,682.0,0.986975,0.340489
2,nationality,575,569.0,558.0,0.980668,0.278582
3,japanese,523,477.0,474.0,0.993711,0.236645
4,chinese,505,462.0,457.0,0.989177,0.228158
5,caucasian,466,451.0,444.0,0.984479,0.221667
...,...,...,...,...,...,...
28,mulato,3,2.0,0.0,0.000000,0.000000
23,hebrew,9,1.0,0.0,0.000000,0.000000
27,persons not of,4,1.0,1.0,1.000000,0.001715
29,turkish,1,1.0,0.0,0.000000,0.000000


In [56]:
grouped_stats_by_doc = combined_stats_by_doc[[
    'Term', 'Retired subjects', 'Confirmed covenants'
]].groupby('Term').agg('sum').reset_index()
grouped_stats_by_doc['Confirmation rate'] = grouped_stats_by_doc['Confirmed covenants'] / grouped_stats_by_doc['Retired subjects']
grouped_stats_by_doc = grouped_stats_by_doc.sort_values(['Confirmed covenants'], ascending=False)
grouped_stats_by_doc['Retired subjects'] = grouped_stats_by_doc['Retired subjects'].apply(lambda x: f"{int(x):,}")
grouped_stats_by_doc['Confirmed covenants'] = grouped_stats_by_doc['Confirmed covenants'].apply(lambda x: f"{int(x):,}")
grouped_stats_by_doc['Confirmation rate'] = grouped_stats_by_doc['Confirmation rate'].apply(lambda x: f"{x:.1%}")
grouped_stats_by_doc.reset_index(delete=True, inplace=True)
grouped_stats_by_doc['Rank'] = grouped_stats_by_doc.index + 1
grouped_stats_by_doc


TypeError: reset_index() got an unexpected keyword argument 'delete'

In [53]:
grouped_stats_by_doc.to_csv('frequency_by_doc.csv', index=False)